In [1]:
import os
%pwd

'd:\\Machine_Learning\\Titanic_Pipeline_Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Titanic_Pipeline_Project'

In [3]:
# Creating the entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from titanic.constants import *
from titanic.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([ config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [6]:
# getting the packages to create teh components

import os
import urllib.request as request
import zipfile
from titanic.utils.common import get_size
from titanic.logging import logger

In [7]:
# Creating the components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded? with the following \n {headers}')
        
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")
    

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)    

In [8]:
# Creating the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-12-28 13:48:00,292: INFO: common: yaml file config\config.yaml loaded successfully]
[2023-12-28 13:48:00,292: INFO: common: yaml file params.yaml loaded successfully]
[2023-12-28 13:48:00,302: INFO: common: created directory at: artifacts]
[2023-12-28 13:48:00,304: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-28 13:48:01,745: INFO: 1270582752: artifacts/data_ingestion/data.zip downloaded? with the following 
 Connection: close
Content-Length: 22564
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2ea9dfe47a2052f4859e54a8c0e465f34ad3acb0e14c10c1d879422276b13647"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 55E6:5EFB:16DE3A:1C98F3:658D2F25
Accept-Ranges: bytes
Date: Thu, 28 Dec 2023 08:18:00 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10231-MAA
X-Cache: MISS
X